# The Problem to Solve
Here we dive deeper into our nonlinear example and investigate the regularisation we employ.

A reminder - the PDE used in the forward model is 

$$-\nabla\cdot k\nabla u = f$$

with strong (Dirichlet) BC on the boundary

$$u = 0 \ \text{on} \ \Gamma$$

where we assert conductivity $k$ is positive by

$$k = k_0e^q.$$

We have a known RHS $f$, take noisy sparse point measurements of the true value $u_{true}$ of our solution $u$, and wish to infer the log-conductivity $q$.

We solve in 2D on a 32x32 mesh with our solution $u$ and log conductivity $q$ each in CG2 function spaces.

In [ ]:
import firedrake
import firedrake_adjoint
mesh = firedrake.UnitSquareMesh(32, 32)
V = firedrake.FunctionSpace(mesh, family='CG', degree=2)
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)

# Generating a Fake "True" Solution
Before diving into solving this inverse problem, we first need to generate some false "true" solution from which we will sample our "observations".

## Fake $q_{true}$
We start off by generating some random true value of $q_{true}$ for use in the forward model which we then plot

In [ ]:
from firedrake import Constant, cos, sin
import numpy as np
from numpy import pi as π
from numpy import random
import matplotlib.pyplot as plt

seed = 1729
generator = random.default_rng(seed)

degree = 5
x = firedrake.SpatialCoordinate(mesh)

q_true = firedrake.Function(Q)
for k in range(degree):
    for l in range(int(np.sqrt(degree**2 - k**2))):
        Z = np.sqrt(1 + k**2 + l**2)
        ϕ = 2 * π * (k * x[0] + l * x[1])

        A_kl = generator.standard_normal() / Z
        B_kl = generator.standard_normal() / Z
        
        expr = Constant(A_kl) * cos(ϕ) + Constant(B_kl) * sin(ϕ)
        mode = firedrake.interpolate(expr, Q)
        
        q_true += mode
        
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(q_true, axes=axes, shading='gouraud')
fig.colorbar(colors);

## Fake $u_{true}$
Now we compute and plot the true solution $u_{true}$ of the PDE.
We rearrange the PDE to be

$$-\nabla\cdot k_0 e^{q_{true}}\nabla u_{true} - f = 0$$

and enforce the strong (Dirichlet) boundary condition that $u_{true} = 0$ on the domain boundary $\Gamma$.

The weak form of this is then

$$ \int_{\Omega} k_0 e^{q_{true}} \nabla u_{true} \cdot \nabla v \,dx - \int_{\Omega} f v \,dx = 0 \quad \forall v \in V \ \text{where} \ u_{true} \in V$$

We set 

$$k_0 = 0.5$$

and

$$f = 1.0.$$

In [ ]:
from firedrake import exp, inner, grad, dx
u_true = firedrake.Function(V)
v = firedrake.TestFunction(V)
f = Constant(1.0)
k0 = Constant(0.5)
bc = firedrake.DirichletBC(V, 0, 'on_boundary')
F = (k0 * exp(q_true) * inner(grad(u_true), grad(v)) - f * v) * dx
firedrake.solve(F == 0, u_true, bc)

fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(u_true, axes=axes, shading='gouraud')
fig.colorbar(colors);

# Generating Observational Data $u_{obs}$
We will create some pretend set of point measurements $u_{obs}$ of the solution $u_{true}$ on some point cloud.

## Generating a Point Cloud $\Omega_v$

To begin with we create a regular point cloud. 
Unlike in the previous example, we choose not to rotate and truncate our data, instead we offset it in from the sides by rescaling then translating.
In Firedrake the point cloud is a `VertexOnlyMesh`.
We can then plot this on top of our mesh to get a sense of the sparsity.

In [ ]:
num_points_side = 5
δs = np.linspace(0, 1, num_points_side)
X, Y = np.meshgrid(δs, δs)
xs = np.vstack((X.flatten(), Y.flatten())).T
assert(len(xs) == num_points_side**2)

offset = 0.1
scaling_factor = 1-2*offset
w = scaling_factor
h = scaling_factor
S = np.array([
    [w, 0],
    [0, h]
])

xs = xs @ S
xs[:, 0] += offset
xs[:, 1] += offset

point_cloud = firedrake.VertexOnlyMesh(mesh, xs)
# Prove the the point cloud coordinates are correct
assert((point_cloud.coordinates.dat.data_ro == xs).all())

fig, axes = plt.subplots()
axes.set_aspect('equal')
firedrake.triplot(mesh, axes)
axes.scatter(xs[:, 0], xs[:, 1])

## Putting Data on the Point Cloud
We now synthesize the observational data $u_{obs}$ by sampling from $u_{true}$ with firedrake's `Function.at` method and adding noise.
The observational data are stored as $\text{P0DG}$ functions on the point cloud `VertexOnlyMesh` i.e. $u_{obs} \in \text{P0DG}(\Omega_v)$.

In [ ]:
# Generate "observed" data
U = u_true.dat.data_ro[:]
u_range = U.max() - U.min()
signal_to_noise = 20
σ = firedrake.Constant(u_range / signal_to_noise)
ζ = generator.standard_normal(len(xs))
u_obs_vals = np.array(u_true.at(xs)) + float(σ) * ζ

# Store data on the point_cloud
P0DG = firedrake.FunctionSpace(point_cloud, 'DG', 0)
u_obs = firedrake.Function(P0DG)
u_obs.dat.data[:] = u_obs_vals

# The PDE Constrained Optimisation Problem

We can now form our PDE constrained optimisation problem:

Given our forward model, we wish to minimise the objective functional

$$J[u, q] = 
\underbrace{\frac{1}{2}\int_{\Omega_v}\left(\frac{u_{obs} - I(u, \text{P0DG}(\Omega_v))}{\sigma}\right)^2dx}_{\text{model-data misfit}} + 
\underbrace{\frac{\alpha^2}{2}\int_\Omega|\nabla q|^2dx}_{\text{regularization}}$$

given a particular value of our "control" $q$ in the forward model $u$ when considered as $u(q)$. 
I.e. the problem is

$$\min_q \hat{J}[q] = \min_q J[u(q), q].$$

Note we don't need to explicitly form $\hat{J}$ - pyadjoint will do that for us.

The reduced functional is minimised via gradient descent. 
The chain rule gives us that:

$$\frac{d\hat{J}}{dq} = 
\underbrace{\frac{\partial J}{\partial q}}_{\text{calculated analytically}} + 
\underbrace{\frac{\partial J}{\partial u} \frac{du}{dq}.}_{\text{found via solution to adjoint problem automatically with pyadjoint}}$$

$I(u, \text{P0DG}(\Omega_v))$ is the interpolation operation of the solution to the forward model $u$ into our point data space $\text{P0DG}(\Omega_v)$.
Crucially this interpolation is now calculable and is annotated in pyadjoint.
We can therefore calculate $\frac{\partial J}{\partial u}$ and $\frac{\partial J}{\partial q}$ (which are needed for minimising the reduced functional $\hat{J}$) using the chain rule since the annotation gives us $\frac{\partial I(u, \text{P0DG}(\Omega_v))}{\partial u}$ and $\frac{\partial I(u, \text{P0DG}(\Omega_v))}{\partial q}$.

For more see the [pyadjoint/dolfin-adjoint documentation](http://www.dolfin-adjoint.org/en/latest/documentation/index.html) in particular the [mathematical background](http://www.dolfin-adjoint.org/en/latest/documentation/maths/index.html#dolfin-adjoint-mathematical-background).

## Running the Forward Model
We start with an initial guess of $q = 0$ and run the forward model. 
We see that we get very different results to the true solution!

In [ ]:
u = firedrake.Function(V)
q = firedrake.Function(Q)
bc = firedrake.DirichletBC(V, 0, 'on_boundary')
F = (k0 * exp(q) * inner(grad(u), grad(v)) - f * v) * dx
firedrake.solve(F == 0, u, bc)

fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(u, axes=axes, shading='gouraud')
fig.colorbar(colors);

We now form the full objective functional which we will minimise.

In [ ]:
misfit_expr = 0.5 * ((u_obs - firedrake.interpolate(u, P0DG)) / σ)**2
α = firedrake.Constant(0.5)
regularisation_expr = 0.5 * α**2 * inner(grad(q), grad(q))

# Should be able to write firedrake.assemble(misfit + regularisation * dx) but can't yet 
# because of the meshes being different
J = firedrake.assemble(misfit_expr * dx) + firedrake.assemble(regularisation_expr * dx)

Next we use firedrake's pyadjoint interface to select our control variable $q$ and form the reduced form of the objective functional $\hat{J}$.

In [ ]:
q̂ = firedrake_adjoint.Control(q)
Ĵ = firedrake_adjoint.ReducedFunctional(J, q̂)

Now we minimise the reduced objective functional.
This takes a few minutes.

In [ ]:
q_min = firedrake_adjoint.minimize(Ĵ, method='Newton-CG', options={'disp': True})

Let's summarise the results.

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=2, sharex=True, sharey=True, figsize=(20,12), dpi=200)
plt.suptitle('Estimating Log-Conductivity $q$ \n\
where $k = k_0e^q$ and $-\\nabla \\cdot k \\nabla u = f$ for known $f$', fontsize=25)
for ax in axes.ravel():
    ax.set_aspect('equal')
#     ax.get_xaxis().set_visible(False)

axes[0, 0].set_title('$u_{true}$', fontsize=25)
colors = firedrake.tripcolor(u_true, axes=axes[0, 0], shading='gouraud')
fig.colorbar(colors, ax=axes[0, 0])
axes[1, 0].set_title('Sampled Noisy $u_{obs}$', fontsize=25)
colors = axes[1, 0].scatter(xs[:, 0], xs[:, 1], c=u_obs_vals)
fig.colorbar(colors, ax=axes[1, 0])

kw = {'vmin': -5, 'vmax': +5, 'shading': 'gouraud'}
axes[0, 1].set_title('$q_{true}$', fontsize=25)
colors = firedrake.tripcolor(q_true, axes=axes[0, 1], **kw)
fig.colorbar(colors, ax=axes[0, 1])
axes[1, 1].set_title('Estimated $q_{est}$ from $u_{obs}$', fontsize=25)
colors = firedrake.tripcolor(q_min, axes=axes[1, 1], **kw);
fig.colorbar(colors, ax=axes[1, 1])

axes[0, 2].axis('off')
q_err = firedrake.Function(Q).assign(q_min-q_true)
l2norm = firedrake.norm(q_err, "L2")
axes[1, 2].set_title('$q_{est}$ - $q_{true}$', fontsize=25)
axes[1, 2].text(0.5, 0.5, f'$L^2$ Norm {l2norm:.2f}', ha='center', fontsize=20)
colors = firedrake.tripcolor(q_err, axes=axes[1, 2], **kw);
fig.colorbar(colors, ax=axes[1, 2])

fig.text(0.5,0.05,r'Functional minimised: $J[u, q] = \frac{1}{2}\int_{\Omega_v}\left(\frac{u_{obs} - I(u, \mathrm{P0DG}(\Omega_v))}{\sigma}\right)^2dx + \frac{\alpha^2}{2}\int_\Omega|\nabla q|^2dx$', ha='center', va='center', fontsize=20)

plt.savefig('poisson-inverse-conductivity-default-summary.png')

# Investigation 1: Changing the form of J

## A) Getting rid of the halves
Let's see what happens if we get rid of our halves from J, i.e.

$$J[u, q] = 
\underbrace{\int_{\Omega_v}\left(\frac{u_{obs} - I(u, \text{P0DG}(\Omega_v))}{\sigma}\right)^2dx}_{\text{model-data misfit}} + 
\underbrace{\alpha^2\int_\Omega|\nabla q|^2dx}_{\text{regularization}}$$

In [ ]:
u = firedrake.Function(V)
q = firedrake.Function(Q)
bc = firedrake.DirichletBC(V, 0, 'on_boundary')
F = (k0 * exp(q) * inner(grad(u), grad(v)) - f * v) * dx
firedrake.solve(F == 0, u, bc)

In [ ]:
misfit_expr = ((u_obs - firedrake.interpolate(u, P0DG)) / σ)**2
α = firedrake.Constant(0.5)
regularisation_expr = α**2 * inner(grad(q), grad(q))

# Should be able to write firedrake.assemble(misfit + regularisation * dx) but can't yet 
# because of the meshes being different
J = firedrake.assemble(misfit_expr * dx) + firedrake.assemble(regularisation_expr * dx)

In [ ]:
q̂ = firedrake_adjoint.Control(q)
Ĵ = firedrake_adjoint.ReducedFunctional(J, q̂)

In [ ]:
q_min = firedrake_adjoint.minimize(Ĵ, method='Newton-CG', options={'disp': True})

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=2, sharex=True, sharey=True, figsize=(20,12), dpi=200)
plt.suptitle('Estimating Log-Conductivity $q$ \n\
where $k = k_0e^q$ and $-\\nabla \\cdot k \\nabla u = f$ for known $f$', fontsize=25)
for ax in axes.ravel():
    ax.set_aspect('equal')
#     ax.get_xaxis().set_visible(False)

axes[0, 0].set_title('$u_{true}$', fontsize=25)
colors = firedrake.tripcolor(u_true, axes=axes[0, 0], shading='gouraud')
fig.colorbar(colors, ax=axes[0, 0])
axes[1, 0].set_title('Sampled Noisy $u_{obs}$', fontsize=25)
colors = axes[1, 0].scatter(xs[:, 0], xs[:, 1], c=u_obs_vals)
fig.colorbar(colors, ax=axes[1, 0])

kw = {'vmin': -5, 'vmax': +5, 'shading': 'gouraud'}
axes[0, 1].set_title('$q_{true}$', fontsize=25)
colors = firedrake.tripcolor(q_true, axes=axes[0, 1], **kw)
fig.colorbar(colors, ax=axes[0, 1])
axes[1, 1].set_title('Estimated $q_{est}$ from $u_{obs}$', fontsize=25)
colors = firedrake.tripcolor(q_min, axes=axes[1, 1], **kw);
fig.colorbar(colors, ax=axes[1, 1])

axes[0, 2].axis('off')
q_err = firedrake.Function(Q).assign(q_min-q_true)
l2norm = firedrake.norm(q_err, "L2")
axes[1, 2].set_title('$q_{est}$ - $q_{true}$', fontsize=25)
axes[1, 2].text(0.5, 0.5, f'$L^2$ Norm {l2norm:.2f}', ha='center', fontsize=20)
colors = firedrake.tripcolor(q_err, axes=axes[1, 2], **kw);
fig.colorbar(colors, ax=axes[1, 2])

fig.text(0.5,0.05,r'Functional minimised: $J[u, q] = \int_{\Omega_v}\left(\frac{u_{obs} - I(u, \mathrm{P0DG}(\Omega_v))}{\sigma}\right)^2dx + \alpha^2\int_\Omega|\nabla q|^2dx$', ha='center', va='center', fontsize=20)

plt.savefig('poisson-inverse-conductivity-nohalves-summary.png')

### A) Conclusion - It doesn't matter
The error is the same!
This makes sense - it's a common factor to both terms in $J$ so effects it's total magnitude but not what will minimise it.

## B) Moving $\sigma$ to the Regularisation

I'm not sure what the point of dividing by the variance $\sigma^2$ in the misfit term is.
Let's try this form of $J$:

$$J[u, q] = 
\underbrace{\int_{\Omega_v}\left(u_{obs} - I(u, \text{P0DG}(\Omega_v))\right)^2dx}_{\text{model-data misfit}} + 
\underbrace{\sigma^2 \alpha^2\int_\Omega|\nabla q|^2dx}_{\text{regularization}}$$

This _should_ give me the same result as before. Let's see.

In [ ]:
u = firedrake.Function(V)
q = firedrake.Function(Q)
bc = firedrake.DirichletBC(V, 0, 'on_boundary')
F = (k0 * exp(q) * inner(grad(u), grad(v)) - f * v) * dx
firedrake.solve(F == 0, u, bc)

misfit_expr = (u_obs - firedrake.interpolate(u, P0DG))**2
α = firedrake.Constant(0.5)
regularisation_expr = σ**2 * α**2 * inner(grad(q), grad(q))

# Should be able to write firedrake.assemble(misfit + regularisation * dx) but can't yet 
# because of the meshes being different
J = firedrake.assemble(misfit_expr * dx) + firedrake.assemble(regularisation_expr * dx)

q̂ = firedrake_adjoint.Control(q)
Ĵ = firedrake_adjoint.ReducedFunctional(J, q̂)

q_min = firedrake_adjoint.minimize(Ĵ, method='Newton-CG', options={'disp': True})

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=2, sharex=True, sharey=True, figsize=(20,12), dpi=200)
plt.suptitle('Estimating Log-Conductivity $q$ \n\
where $k = k_0e^q$ and $-\\nabla \\cdot k \\nabla u = f$ for known $f$', fontsize=25)
for ax in axes.ravel():
    ax.set_aspect('equal')
#     ax.get_xaxis().set_visible(False)

axes[0, 0].set_title('$u_{true}$', fontsize=25)
colors = firedrake.tripcolor(u_true, axes=axes[0, 0], shading='gouraud')
fig.colorbar(colors, ax=axes[0, 0])
axes[1, 0].set_title('Sampled Noisy $u_{obs}$', fontsize=25)
colors = axes[1, 0].scatter(xs[:, 0], xs[:, 1], c=u_obs_vals)
fig.colorbar(colors, ax=axes[1, 0])

kw = {'vmin': -5, 'vmax': +5, 'shading': 'gouraud'}
axes[0, 1].set_title('$q_{true}$', fontsize=25)
colors = firedrake.tripcolor(q_true, axes=axes[0, 1], **kw)
fig.colorbar(colors, ax=axes[0, 1])
axes[1, 1].set_title('Estimated $q_{est}$ from $u_{obs}$', fontsize=25)
colors = firedrake.tripcolor(q_min, axes=axes[1, 1], **kw);
fig.colorbar(colors, ax=axes[1, 1])

axes[0, 2].axis('off')
q_err = firedrake.Function(Q).assign(q_min-q_true)
l2norm = firedrake.norm(q_err, "L2")
axes[1, 2].set_title('$q_{est}$ - $q_{true}$', fontsize=25)
axes[1, 2].text(0.5, 0.5, f'$L^2$ Norm {l2norm:.2f}', ha='center', fontsize=20)
colors = firedrake.tripcolor(q_err, axes=axes[1, 2], **kw);
fig.colorbar(colors, ax=axes[1, 2])

fig.text(0.5,0.05,r'Functional minimised: $J[u, q] = \int_{\Omega_v}\left(u_{obs} - I(u, \mathrm{P0DG}(\Omega_v))\right)^2dx + \sigma^2 \alpha^2\int_\Omega|\nabla q|^2dx$', ha='center', va='center', fontsize=20)

plt.savefig('poisson-inverse-conductivity-moved-sigma-summary.png')

Basically the same, though noteably took fewer iterations to minimise!